In [1]:
import pandas as pd
import numpy as np

# 一、2002 年-2018 年上海机动车拍照拍卖

In [51]:
df = pd.read_csv('./数据集/2002年-2018年上海机动车拍照拍卖.csv')
df.head()

,Date,Total number of license issued,lowest price,avg price,Total number of applicants
0,2-Jan,1400,13600,14735,3718
1,2-Feb,1800,13100,14057,4590
2,2-Mar,2000,14300,14662,5190
3,2-Apr,2300,16000,16334,4806
4,2-May,2350,17800,18357,4665


### (1) 哪一次拍卖的中标率首次小于 5%?

In [18]:
df['rate'] = df['Total number of license issued'] / df['Total number of applicants']
df.query('rate<0.05')['Date'].iloc[0]

'15-May'

### (2) 按年统计拍卖最低价的下列统计量:最大值、均值、0.75 分位数，要求显示在同一张表上。

In [33]:
year = [i[0] for i in df.Date.str.split('-')]
res = df.groupby(year)['lowest price '].agg(['max','mean',lambda x:x.quantile(0.75)])
res.rename(columns={res.columns[2]:'quantile75'},inplace = True)
res.head()

,max,mean,quantile75
10,44900,38008.333333,41825
11,53800,47958.333333,51000
12,68900,61108.333333,65325
13,90800,79125.000000,82550
14,74600,73816.666667,74000


### (3) 将第一列时间列拆分成两个列，一列为年份(格式为 20××)，另一列为 月份(英语缩写)，添加到列表作为第一第二列，并将原表第一列删除， 其他列依次向后顺延。

In [52]:
year = [int(i[0])+2000 for i in df.Date.str.split('-')]
mon = [i[1] for i in df.Date.str.split('-')]
df['year'],df['month'] = year,mon
df.head()

,Date,Total number of license issued,lowest price,avg price,Total number of applicants,year,month
0,2-Jan,1400,13600,14735,3718,2002,Jan
1,2-Feb,1800,13100,14057,4590,2002,Feb
2,2-Mar,2000,14300,14662,5190,2002,Mar
3,2-Apr,2300,16000,16334,4806,2002,Apr
4,2-May,2350,17800,18357,4665,2002,May


In [53]:
del df['Date']
cols = df.columns.tolist()

In [54]:
res3 = df.reindex(columns=cols[-2:]+cols[:-2])
res3.head()

,year,month,Total number of license issued,lowest price,avg price,Total number of applicants
0,2002,Jan,1400,13600,14735,3718
1,2002,Feb,1800,13100,14057,4590
2,2002,Mar,2000,14300,14662,5190
3,2002,Apr,2300,16000,16334,4806
4,2002,May,2350,17800,18357,4665


### (4) 现在将表格行索引设为多级索引，外层为年份，内层为原表格第二至第 五列的变量名，列索引为月份。

In [57]:
res3.set_index(['year']+cols[:4]).head()

month
year Total number of license issued lowest price  avg price Total number of applicants      
2002 1400                           13600         14735     3718                         Jan
     1800                           13100         14057     4590                         Feb
     2000                           14300         14662     5190                         Mar
     2300                           16000         16334     4806                         Apr
     2350                           17800         18357     4665                         May

### (5)一般而言某个月最低价与上月最低价的差额，会与该月均值与上月均值 的差额具有相同的正负号，哪些拍卖时间不具有这个特点?

In [62]:
df.head(2)

,Total number of license issued,lowest price,avg price,Total number of applicants,year,month
0,1400,13600,14735,3718,2002,Jan
1,1800,13100,14057,4590,2002,Feb


In [64]:
tmp = df.set_index(['year','month']).iloc[:,[1,2]]
tmp.head(2)

lowest price   avg price
year month                          
2002 Jan            13600      14735
     Feb            13100      14057

In [77]:
tmp1 = tmp.diff().iloc[:,0]*tmp.diff().iloc[:,1]
tmp1[tmp1<0]

year  month
2003  Oct     -15544000.0
      Nov       -167400.0
2004  Jun     -92575000.0
2005  Jan      -1790400.0
      Feb       -304000.0
      Sep     -10467000.0
2006  May        -37400.0
      Sep      -3311800.0
2007  Jan       -592800.0
      Feb       -300600.0
      Dec      -6555000.0
2012  Oct       -141500.0
dtype: float64

### (6)将某一个月牌照发行量与其前两个月发行量均值的差额定义为发行增 益，最初的两个月用 0 填充，求发行增益极值出现的时间。


In [82]:
df = pd.read_csv('./数据集/2002年-2018年上海机动车拍照拍卖.csv')
gain = df['Total number of license issued']-df['Total number of license issued'].rolling(2).mean().shift(1)
df['gain'] = gain.fillna(0)

In [84]:
df.set_index('Date',inplace=True)
print('极大值：',df.gain.idxmax())
print('极小值：',df.gain.idxmin())

极大值： 8-Jan
极小值： 8-Apr


# 二、2007 年-2019 年俄罗斯机场货运航班运载量


In [85]:
df = pd.read_csv('./数据集/2007年-2019年俄罗斯货运航班运载量.csv')
df.head()

,Airport name,Year,January,February,March,April,May,June,July,August,September,October,November,December,Whole year,Airport coordinates
0,Abakan,2019,44.70,66.21,72.7,75.82,100.34,78.38,63.88,73.06,66.74,75.44,110.5,89.8,917.57,"(Decimal('91.399735'), Decimal('53.751351'))"
1,Aikhal,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('111.543324'), Decimal('65.957161'))"
2,Loss,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('125.398355'), Decimal('58.602489'))"
3,Amderma,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('61.577429'), Decimal('69.759076'))"
4,Anadyr (Carbon),2019,81.63,143.01,260.9,304.36,122.00,106.87,84.99,130.00,102.00,118.00,94.0,199.0,1746.76,"(Decimal('177.738273'), Decimal('64.713433'))"


### (1) 求每年货运航班总运量。

In [86]:
df.groupby('Year')['Whole year'].sum()

Year
2007    659438.23
2008    664682.46
2009    560809.77
2010    693033.98
2011    818691.71
2012    846388.03
2013    792337.08
2014    729457.12
2015    630208.97
2016    679370.15
2017    773662.28
2018    767095.28
2019    764606.27
Name: Whole year, dtype: float64

### (2) 每年记录的机场都是相同的吗?

In [90]:
if df.groupby('Year')['Airport name'].count().nunique() == 1:
    print('Yes')
else:
    print('No')

No


### (3) 按年计算 2010 年-2015 年全年货运量记录为 0 的机场航班比例。

In [94]:
df_ = df[df.Year.isin(range(2010,2016))]

In [108]:
df_.groupby('Year')['Whole year'].agg(lambda x:x.value_counts().get(0)/len(x))

Year
2010    0.767123
2011    0.770548
2012    0.770548
2013    0.770548
2014    0.770548
2015    0.770548
Name: Whole year, dtype: float64

### (4) 若某机场至少存在 5 年或以上满足所有月运量记录都为 0，则将其所有年份的记录信息从表中删除，并返回处理后的表格


In [116]:
df_ = df[(df.loc[:,'January':'December'] == 0).all(axis = 1)]

In [140]:
tmp = df_.groupby('Airport name')['Year'].count()
airp = tmp[tmp>5].index.tolist()
res = df[~df['Airport name'].isin(airp)]
res.head()

,Airport name,Year,January,February,March,April,May,June,July,August,September,October,November,December,Whole year,Airport coordinates
0,Abakan,2019,44.70,66.21,72.70,75.82,100.34,78.38,63.88,73.06,66.74,75.44,110.50,89.80,917.57,"(Decimal('91.399735'), Decimal('53.751351'))"
4,Anadyr (Carbon),2019,81.63,143.01,260.90,304.36,122.00,106.87,84.99,130.00,102.00,118.00,94.00,199.00,1746.76,"(Decimal('177.738273'), Decimal('64.713433'))"
5,Anapa (Vitjazevo),2019,45.92,53.15,54.00,54.72,52.00,67.45,172.31,72.57,70.00,63.00,69.00,82.10,856.22,"(Decimal('37.341511'), Decimal('45.003748'))"
8,Arkhangelsk (Talagy),2019,85.61,118.70,131.39,144.82,137.95,140.18,128.56,135.68,124.75,139.60,210.27,307.10,1804.61,"(Decimal('40.714892'), Decimal('64.596138'))"
9,Astrakhan (Narimanovo),2019,51.75,61.08,65.60,71.84,71.38,63.95,164.86,79.46,85.21,87.23,79.06,99.16,980.58,"(Decimal('47.999896'), Decimal('46.287344'))"


### (5) 采用一种合理的方式将所有机场划分为东南西北四个分区，并给出 2017年-2019 年货运总量最大的区域。

In [152]:
coor[0][1]

'53.751351'

In [222]:
ll = df['Airport coordinates'].str.extractall(r"(\d+\.?\d*)").reset_index().pivot(index = 'level_0',columns = 'match',values = 0).astype('float')
ll.columns = ['lati','long']
ll.describe()

,lati,long
count,3533.000000,3533.000000
mean,89.690988,58.548133
std,43.121096,7.825598
min,6.226888,8.607871
25%,49.298591,53.690803
50%,82.667491,58.381633
75%,129.750225,65.026271
max,179.380105,73.516386


In [237]:
ll = ll.assign(lati_cut = pd.cut(ll.lati,[0,90,180],labels = ['西','东']),\
        long_cut = pd.cut(ll.long,[0,45,90],labels = ['北','南']))

df['direction'] = ll.lati_cut.str.cat(ll.long_cut)
df.head()

,Airport name,Year,January,February,March,April,May,June,July,August,September,October,November,December,Whole year,Airport coordinates,direction
0,Abakan,2019,44.70,66.21,72.7,75.82,100.34,78.38,63.88,73.06,66.74,75.44,110.5,89.8,917.57,"(Decimal('91.399735'), Decimal('53.751351'))",东南
1,Aikhal,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('111.543324'), Decimal('65.957161'))",东南
2,Loss,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('125.398355'), Decimal('58.602489'))",东南
3,Amderma,2019,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,"(Decimal('61.577429'), Decimal('69.759076'))",西南
4,Anadyr (Carbon),2019,81.63,143.01,260.9,304.36,122.00,106.87,84.99,130.00,102.00,118.00,94.0,199.0,1746.76,"(Decimal('177.738273'), Decimal('64.713433'))",东南


In [243]:
df[df.Year.isin(range(2017,2020))].groupby(['Year','direction'])['Whole year'].sum().groupby('Year').idxmax()

Year
2017    (2017, 西南)
2018    (2018, 西南)
2019    (2019, 西南)
Name: Whole year, dtype: object

### (6) 在统计学中常常用秩代表排名，现在规定某个机场某年某个月的秩为该 机场该月在当年所有月份中货运量的排名(例如 *** 机场 19 年 1 月运 量在整个 19 年 12 个月中排名第一，则秩为 1)，那么判断某月运量情 况的相对大小的秩方法为将所有机场在该月的秩排名相加，并将这个量 定义为每一个月的秩综合指数，请根据上述定义计算 2016 年 12 个月 的秩综合指数。

In [205]:
df_ = df.iloc[:,:-2].query('Year == 2016').copy()
df_.update(df_.loc[:,'January':'December'].apply(lambda x:x.rank(method = 'min'),axis = 1))

In [207]:
df_.set_index('Airport name',inplace=True)
del df_['Year']
df_.sum()

January      390.0
February     495.0
March        616.0
April        689.0
May          619.0
June         621.0
July         591.0
August       691.0
September    724.0
October      759.0
November     812.0
December     893.0
dtype: float64

# 三、新冠肺炎在美国的传播

In [244]:
death = pd.read_csv('./数据集/美国死亡数.csv')
confirmed = pd.read_csv('./数据集/美国确证数.csv')

In [245]:
death.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2020/4/17,2020/4/18,2020/4/19,2020/4/20,2020/4/21,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,...,2,2,2,1,1,2,2,2,2,2
1,84001003,US,USA,840,1003,Baldwin,Alabama,US,30.727750,-87.722071,...,2,2,2,3,3,3,3,3,3,3
2,84001005,US,USA,840,1005,Barbour,Alabama,US,31.868263,-85.387129,...,0,0,0,0,0,0,0,0,0,0
3,84001007,US,USA,840,1007,Bibb,Alabama,US,32.996421,-87.125115,...,0,0,0,0,0,0,0,0,0,0
4,84001009,US,USA,840,1009,Blount,Alabama,US,33.982109,-86.567906,...,0,0,0,0,0,0,0,0,0,0


In [246]:
confirmed.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2020/4/17,2020/4/18,2020/4/19,2020/4/20,2020/4/21,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26
0,84001001,US,USA,840,1001,Autauga,Alabama,US,32.539527,-86.644082,...,26,25,26,28,30,32,33,36,36,37
1,84001003,US,USA,840,1003,Baldwin,Alabama,US,30.727750,-87.722071,...,103,109,112,117,123,132,143,147,147,161
2,84001005,US,USA,840,1005,Barbour,Alabama,US,31.868263,-85.387129,...,15,18,20,22,28,29,30,32,32,33
3,84001007,US,USA,840,1007,Bibb,Alabama,US,32.996421,-87.125115,...,24,26,28,32,32,34,33,34,34,38
4,84001009,US,USA,840,1009,Blount,Alabama,US,33.982109,-86.567906,...,20,20,21,22,26,29,31,31,31,34


### (1) 用 corr() 函数计算县(每行都是一个县)人口与表中最后一天记录日期 死亡数的相关系数。

In [276]:
death.Population.corr(death.iloc[:,-1])

0.4038441973480701

### (2) 截止到 4 月 1 日，统计每个州零感染县的比例。

In [272]:
confirmed.loc[:,:'2020/4/1'].groupby('Province_State').apply(lambda x:(x.loc[:,'2020/1/22':'2020/4/1'].sum(axis = 1)==0).sum()/len(x))

Province_State
Alabama                 0.104478
Alaska                  0.793103
Arizona                 0.000000
Arkansas                0.293333
California              0.137931
Colorado                0.218750
Connecticut             0.000000
Delaware                0.000000
District of Columbia    0.000000
Florida                 0.164179
Georgia                 0.119497
Hawaii                  0.200000
Idaho                   0.386364
Illinois                0.450980
Indiana                 0.108696
Iowa                    0.404040
Kansas                  0.600000
Kentucky                0.433333
Louisiana               0.046875
Maine                   0.187500
Maryland                0.041667
Massachusetts           0.142857
Michigan                0.168675
Minnesota               0.356322
Mississippi             0.060976
Missouri                0.391304
Montana                 0.625000
Nebraska                0.741935
Nevada                  0.470588
New Hampshire           0.00

### (3) 请找出最早出确证病例的三个县。

In [419]:
tmp = (confirmed.loc[:,'2020/1/22':]>0).astype('int')#.idxmax(axis = 1).sort_values()
tmp = tmp[~ (tmp == 0).all(axis = 1)]

In [418]:
tmp.idxmax(axis = 1).sort_values().index[:3]

Int64Index([84053033, 84017031, 84006037], dtype='int64', name='UID')

### (4) 按州统计单日死亡增加数，并给出哪个州在哪一天确诊数增加最大(这 里指的是在所有州和所有天两个指标一起算，不是分别算)。

In [307]:
state = death.loc[:,'2020/1/22':].groupby(death['Province_State']).sum()
add = state.diff(axis = 1)
add.head()

,2020/1/22,2020/1/23,2020/1/24,2020/1/25,2020/1/26,2020/1/27,2020/1/28,2020/1/29,2020/1/30,2020/1/31,...,2020/4/17,2020/4/18,2020/4/19,2020/4/20,2020/4/21,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,5.0,4.0,6.0,20.0,13.0,6.0,7.0,0.0,10.0
Alaska,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arizona,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,39.0,3.0,7.0,18.0,23.0,18.0,17.0,7.0,2.0
Arkansas,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,2.0,1.0,0.0,3.0,2.0,1.0,3.0
California,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,81.0,103.0,35.0,48.0,57.0,139.0,112.0,88.0,68.0,36.0


In [316]:
confirmed.loc[:,'2020/1/22':].groupby(confirmed['Province_State']).sum().diff(axis = 1).stack().idxmax()

('New York', '2020/4/15')

### (5) 现需对每个州编制确证与死亡表，第一列为时间，并且起始时间为该州 开始出现死亡比例的那一天，第二列和第三列分别为确证数和死亡数， 每个州需要保存为一个单独的 csv 文件，文件名为“州名.csv”。


In [317]:
s_death = death.loc[:,'2020/1/22':].groupby(death['Province_State']).sum()
s_con = confirmed.loc[:,'2020/1/22':].groupby(confirmed['Province_State']).sum()

In [320]:
states = s_death.index.tolist()

In [347]:
def make_table(state,path):
    tocsv = pd.DataFrame()
    s = s_death.loc[state]
    c = s_con.loc[state]
    start = (s!=0).idxmax()
    tocsv = tocsv.assign(确证数= c[start:],
                死亡数 = s[start:])
    tocsv = tocsv.reset_index().rename(columns = {'index':'时间'})
    tocsv.to_csv(path,index = False)
    print('%s已写入'%state)
    return 

In [348]:
import os
path = './result/州确证死亡表'
if not os.path.exists(path):
    os.makedirs(path)
else:
    for i in states:
        name = i+'.csv'
        make_table(i,os.path.join(path,name))

Alabama已写入
Alaska已写入
Arizona已写入
Arkansas已写入
California已写入
Colorado已写入
Connecticut已写入
Delaware已写入
District of Columbia已写入
Florida已写入
Georgia已写入
Hawaii已写入
Idaho已写入
Illinois已写入
Indiana已写入
Iowa已写入
Kansas已写入
Kentucky已写入
Louisiana已写入
Maine已写入
Maryland已写入
Massachusetts已写入
Michigan已写入
Minnesota已写入
Mississippi已写入
Missouri已写入
Montana已写入
Nebraska已写入
Nevada已写入
New Hampshire已写入
New Jersey已写入
New Mexico已写入
New York已写入
North Carolina已写入
North Dakota已写入
Ohio已写入
Oklahoma已写入
Oregon已写入
Pennsylvania已写入
Rhode Island已写入
South Carolina已写入
South Dakota已写入
Tennessee已写入
Texas已写入
Utah已写入
Vermont已写入
Virginia已写入
Washington已写入
West Virginia已写入
Wisconsin已写入
Wyoming已写入


In [349]:
pd.read_csv('./result/州确证死亡表/California.csv').head()

,时间,确证数,死亡数
0,2020/3/4,35,1
1,2020/3/5,51,1
2,2020/3/6,59,1
3,2020/3/7,81,1
4,2020/3/8,95,1


### (6) 现需对 4 月 1 日至 4 月 10 日编制新增确证数与新增死亡数表，第一列 为州名，第二列和第三列分别为新增确证数和新增死亡数，分别保存为 十个单独的 csv 文件，文件名为“日期.csv”。

In [359]:
d1to10 = s_death.loc[:,'2020/3/31':'2020/4/10'].diff(axis = 1).dropna(axis = 1)
c1to10 = s_con.loc[:,'2020/3/31':'2020/4/10'].diff(axis = 1).dropna(axis = 1)

In [361]:
d1to10.head()

,2020/4/1,2020/4/2,2020/4/3,2020/4/4,2020/4/5,2020/4/6,2020/4/7,2020/4/8,2020/4/9,2020/4/10
Province_State,,,,,,,,,,
Alabama,12.0,0.0,11.0,6.0,1.0,4.0,15.0,2.0,4.0,10.0
Alaska,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
Arizona,2.0,18.0,0.0,18.0,4.0,1.0,4.0,18.0,3.0,0.0
Arkansas,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0
California,25.0,40.0,25.0,26.0,59.0,32.0,52.0,61.0,49.0,41.0


In [382]:
dates = d1to10.columns.tolist()
concat = pd.concat([c1to10,d1to10],keys=['新增确证数','新增死亡数'],axis = 1)
concat = concat.swaplevel(axis = 1).sort_index(axis = 1)
concat.head()

2020/4/1         2020/4/10         2020/4/2         2020/4/3  \
                  新增死亡数   新增确证数     新增死亡数   新增确证数    新增死亡数   新增确证数    新增死亡数   
Province_State                                                                
Alabama            12.0    73.0      10.0   244.0      0.0   173.0     11.0   
Alaska              0.0    13.0       0.0    11.0      0.0    11.0      0.0   
Arizona             2.0   124.0       0.0    94.0     18.0   183.0      0.0   
Arkansas            1.0    51.0       0.0    44.0      0.0    60.0      0.0   
California         25.0  1189.0      41.0  1371.0     40.0  1374.0     25.0   

                       2020/4/4        2020/4/5         2020/4/6         \
                 新增确证数    新增死亡数  新增确证数    新增死亡数   新增确证数    新增死亡数  新增确证数   
Province_State                                                            
Alabama          262.0      6.0  119.0      1.0   151.0      4.0  187.0   
Alaska            14.0      0.0   14.0      0.0    14.0      0.0    5.0   
Arizona          173.0     18.0  250.0      4.0   250.0      1.0  191.0   
Arkansas          57.0      0.0   33.0      0.0    90.0      0.0   28.0   
California      1231.0     26.0  833.0     59.0  2197.0     32.0  942.0   

               2020/4/7         2020/4/8         2020/4/9         
                  新增死亡数   新增确证数    新增死亡数   新增确证数    新增死亡数  新增确证数  
Province_State                                                    
Alabama            15.0   217.0      2.0   159.0      4.0  375.0  
Alaska              4.0    23.0      0.0    12.0      0.0   10.0  
Arizona             4.0   115.0     18.0   151.0      3.0  292.0  
Arkansas            0.0    94.0      0.0    54.0     15.0  150.0  
California         52.0  1333.0     61.0  1421.0     49.0  976.0

In [388]:
path = './result/4月1日至4月10日新增确证数与新增死亡数表'
if not os.path.exists(path):
    os.makedirs(path)
for d in dates:
    name = os.path.join(path,d.replace('/','-')+'.csv')
    concat[d].reset_index().to_csv(name,index = False)
    print(name,'已写入')

./result/4月1日至4月10日新增确证数与新增死亡数表/2020-4-1.csv 已写入
./result/4月1日至4月10日新增确证数与新增死亡数表/2020-4-2.csv 已写入
./result/4月1日至4月10日新增确证数与新增死亡数表/2020-4-3.csv 已写入
./result/4月1日至4月10日新增确证数与新增死亡数表/2020-4-4.csv 已写入
./result/4月1日至4月10日新增确证数与新增死亡数表/2020-4-5.csv 已写入
./result/4月1日至4月10日新增确证数与新增死亡数表/2020-4-6.csv 已写入
./result/4月1日至4月10日新增确证数与新增死亡数表/2020-4-7.csv 已写入
./result/4月1日至4月10日新增确证数与新增死亡数表/2020-4-8.csv 已写入
./result/4月1日至4月10日新增确证数与新增死亡数表/2020-4-9.csv 已写入
./result/4月1日至4月10日新增确证数与新增死亡数表/2020-4-10.csv 已写入


In [389]:
pd.read_csv('./result/4月1日至4月10日新增确证数与新增死亡数表/2020-4-2.csv').head()

,Province_State,新增死亡数,新增确证数
0,Alabama,0.0,173.0
1,Alaska,0.0,11.0
2,Arizona,18.0,183.0
3,Arkansas,0.0,60.0
4,California,40.0,1374.0
